In [ ]:
pip install finance-datareader # pip 설치


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.2 MB/s eta 0:00:00


# 기업의 Beta 구하는 방법

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import statsmodels.api as sm
import datetime

# 티커 설정
stock_ticker = '005380.KS'    # 분석할 기업의 티커 (예: 특정 한국 기업)
market_ticker = '^KS11'       # 시장 지수의 티커 (S&P 500)

# 데이터 다운로드 (과거 3년간 주간 데이터)
end_date = datetime.datetime.today()
start_date = end_date - datetime.timedelta(days=5*365)

# 주간 데이터 다운로드: interval='1wk'
stock_data = yf.download(stock_ticker, start=start_date, end=end_date, interval='1wk')
market_data = yf.download(market_ticker, start=start_date, end=end_date, interval='1wk')

# 종가(Close) 추출
stock_close = stock_data['Close']
market_close = market_data['Close']

# 일주일 간의 수익률 계산 (비율로 표현)
stock_returns = stock_close.pct_change().dropna()
market_returns = market_close.pct_change().dropna()

# 두 데이터의 결합 (동일한 날짜로 정렬)
data = pd.concat([stock_returns, market_returns], axis=1).dropna()
data.columns = ['Stock_Return', 'Market_Return']

# 독립 변수에 상수항 추가 (절편)
X = sm.add_constant(data['Market_Return'])  # X 변수: 시장 수익률
Y = data['Stock_Return']                    # Y 변수: 기업 수익률

# 회귀 모델 피팅
model = sm.OLS(Y, X).fit()

# Raw Beta 추출
raw_beta = model.params['Market_Return']
print(f"Raw Beta: {raw_beta:.4f}")

# Adjusted Beta 계산 (Blume 조정법)
adjusted_beta = (2/3) * raw_beta + (1/3) * 1.0
print(f"Adjusted Beta (Blume): {adjusted_beta:.4f}")


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Raw Beta: 1.3088
Adjusted Beta (Blume): 1.2059


#코스피 연평균 수익률

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import datetime

# 티커 설정
market_ticker = '^KS11'  # 코스피 지수의 티커

# 데이터 다운로드 기간 설정 (과거 3년간 주간 데이터)
end_date = datetime.datetime.today()
start_date = end_date - datetime.timedelta(days=5*365)

# 주간 데이터 다운로드: interval='1wk'
market_data = yf.download(market_ticker, start=start_date, end=end_date, interval='1wk')

# 종가(Close) 추출
market_close = market_data['Close']

# 데이터가 정상적으로 다운로드되었는지 확인
if market_close.empty:
    raise ValueError("데이터를 다운로드할 수 없습니다. 티커가 올바른지 확인해주세요.")

# 시작값과 종료값 추출 (숫자형으로 변환)
begin_value = market_close.iloc[0].item()
end_value = market_close.iloc[-1].item()

# 데이터 기간 (연 단위)
number_of_years = 3

# CAGR 계산
cagr = (end_value / begin_value) ** (1 / number_of_years) - 1

# 결과 출력
print(f"시작 종가 (3년 전): {begin_value:.2f} ")
print(f"종료 종가 (현재): {end_value:.2f} ")
print(f"연평균 수익률 (CAGR): {cagr:.2%}")


[*********************100%***********************]  1 of 1 completed

시작 종가 (3년 전): 2132.30 
종료 종가 (현재): 2855.77 
연평균 수익률 (CAGR): 10.23%


# risk free rate (한국 국채 10년물 수익률)

In [ ]:
from datetime import datetime, timedelta
from FinanceDataReader.investing.data import InvestingDailyReader

end = datetime.now().strftime("%Y-%m-%d")
start = (datetime.now() - timedelta(20)).strftime("%Y-%m-%d")

reader = InvestingDailyReader(symbol="KR10YT=RR", start=start, end=end)
data = reader.read()


data['Close'].iloc[-1]

np.float64(2.837)

총 WACC 계산함수

In [16]:
def calculate_wacc(stock_ticker, market_ticker='000660.KS', risk_free_rate=0.03, market_risk_premium=0.05):
    # 데이터 다운로드
    end_date = datetime.today()
    start_date = end_date - timedelta(days=5*365)

    stock_data = yf.download(stock_ticker, start=start_date, end=end_date)
    market_data = yf.download(market_ticker, start=start_date, end=end_date)

    # 종가 추출
    stock_close = stock_data['Close']
    market_close = market_data['Close']

    # 일일 수익률 계산
    stock_returns = stock_close.pct_change().dropna()
    market_returns = market_close.pct_change().dropna()

    # 데이터 결합 및 정렬
    data = pd.concat([stock_returns, market_returns], axis=1).dropna()
    data.columns = ['Stock_Return', 'Market_Return']

    # Raw Beta 계산 (OLS 사용)
    X = sm.add_constant(data['Market_Return'])
    Y = data['Stock_Return']

    model = sm.OLS(Y, X).fit()
    raw_beta = model.params['Market_Return']

    # Adjusted Beta 계산 (Blume 조정법)
    adjusted_beta = (2/3) * raw_beta + (1/3) * 1.0

    # Cost of Equity (Re) 계산
    Re = risk_free_rate + adjusted_beta * market_risk_premium

    # 재무제표 가져오기
    ticker = yf.Ticker(stock_ticker)
    income_statement = ticker.financials
    balance_sheet = ticker.balance_sheet

    # 이자 비용 추출
    if 'Interest Expense' in income_statement.index :
        interest_expense = income_statement.loc['Interest Expense'][0]
        if income_statement.loc['Interest Expense'][0] != True:
          interest_expense = income_statement.loc['Interest Expense'][1]
    elif 'Interest and Other Financing Expenses' in income_statement.index:
        interest_expense = income_statement.loc['Interest and Other Financing Expenses'][0]
    else:
        raise ValueError("이자 비용 항목을 찾을 수 없습니다.")

    # 총 부채 추출
    if 'Total Debt' in balance_sheet.index:
        total_debt = balance_sheet.loc['Total Debt'][0]
    elif 'Long Term Debt' in balance_sheet.index and 'Short Long Term Debt' in balance_sheet.index:
        total_debt = balance_sheet.loc['Long Term Debt'][0] + balance_sheet.loc['Short Long Term Debt'][0]
    else:
        raise ValueError("총 부채 항목을 찾을 수 없습니다.")

    # Cost of Debt (Rd) 계산
    Rd = interest_expense / total_debt

    # 법인세 비용 및 세전 이익 추출
    if 'Income Tax Expense' in income_statement.index:
        income_tax_expense = income_statement.loc['Income Tax Expense'][0]
    elif 'Income Tax Expense Benefit' in income_statement.index:
        income_tax_expense = income_statement.loc['Income Tax Expense Benefit'][0]
    elif 'Tax Provision' in income_statement.index:
        income_tax_expense = income_statement.loc['Tax Provision'][0]
    else:
        raise ValueError("법인세 비용 항목을 찾을 수 없습니다.")

    if 'Income Before Tax' in income_statement.index:
        ebt = income_statement.loc['Income Before Tax'][0]
    elif 'Pretax Income' in income_statement.index:
        ebt = income_statement.loc['Pretax Income'][0]
    else:
        raise ValueError("세전 이익 항목을 찾을 수 없습니다.")

    # Corporate Tax Rate (Tc) 계산
    Tc = income_tax_expense / ebt

    # 자기자본 (Equity, E): 시가총액
    market_cap = ticker.info['marketCap']
    E = market_cap

    # 타인자본 (Debt, D): 총 부채
    D = total_debt

    # 전체 자본 (V = E + D)
    V = E + D

    # 자본 구조 비율 계산
    E_V = E / V
    D_V = D / V

    # WACC 계산
    WACC = (E_V * Re) + (D_V * Rd * (1 - Tc))

    return {
        'Raw Beta': raw_beta,
        'Adjusted Beta': adjusted_beta,
        'Cost of Equity (Re)': Re,
        'Cost of Debt (Rd)': Rd,
        'Corporate Tax Rate (Tc)': Tc,
        'Market Capitalization (E)': E,
        'Total Debt (D)': D,
        'Total Capital (V)': V,
        'Equity Ratio (E/V)': E_V,
        'Debt Ratio (D/V)': D_V,
        'WACC': WACC
    }

# 함수 사용 예시
wacc_info = calculate_wacc('AAPL')
for key, value in wacc_info.items():
    if 'Ratio' in key or 'Tax Rate' in key or 'Cost' in key:
        print(f"{key}: {value:.2%}")
    else:
        print(f"{key}: {value:.4f}")


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
<ipython-input-16-db4b1eec136f>:41: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  interest_expense = income_statement.loc['Interest Expense'][0]
<ipython-input-16-db4b1eec136f>:42: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if income_statement.loc['Interest Expense'][0] != True:
<ipython-input-16-db4b1eec136f>:43: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFra

Raw Beta: 0.0122
Adjusted Beta: 0.3414
Cost of Equity (Re): 4.71%
Cost of Debt (Rd): 3.69%
Corporate Tax Rate (Tc): 24.09%
Market Capitalization (E): 3008816742400.0000
Total Debt (D): 106629000000.0000
Total Capital (V): 3115445742400.0000
Equity Ratio (E/V): 96.58%
Debt Ratio (D/V): 3.42%
WACC: 0.0464


In [36]:
import yfinance as yf
import pandas as pd
import numpy as np

# ----------------------------------------------------------------------
# 0. 파라미터
# ----------------------------------------------------------------------
TICKER   = "AAPL"
LOOKBACK = 5           # 최근 5개 회계연도 사용
YEARS_FC = 5           # 앞으로 예측할 연도 수
WACC     = 0.09
G_TERMINAL = 0.02

# ----------------------------------------------------------------------
# 1. yfinance → 핵심 항목 5줄짜리 DataFrame 만들기
# ----------------------------------------------------------------------
def grab_row(df, *labels):
    """df.index 안 첫 번째 존재 라벨 Series 반환 (없으면 NaN Series)."""
    for lab in labels:
        if lab in df.index:
            return df.loc[lab]
    return pd.Series([np.nan] * df.shape[1], index=df.columns)

tk   = yf.Ticker(TICKER)
inc  = tk.financials.iloc[:, :LOOKBACK].iloc[:, ::-1]   # Income Statement
cfs  = tk.cashflow.iloc[:, :LOOKBACK].iloc[:, ::-1]     # Cash-flow Statement

df = pd.DataFrame({
    "revenue"    : grab_row(inc, "Total Revenue", "Revenue"),
    "ebit"       : grab_row(inc, "Operating Income", "EBIT"),
    "tax"        : grab_row(inc, "Tax Provision", "Income Tax Expense"),
    "dep"        : grab_row(cfs, "Depreciation And Amortization",
                                  "Depreciation & Amortization",
                                  "Depreciation"),
    "nwc_change" : grab_row(cfs, "Change In Working Capital",
                                  "Change In Operating Assets And Liabilities")
}).astype(float)

# 결측치 많으면 드롭
df = df.dropna(how="all")

# ----------------------------------------------------------------------
# 2. 핵심 비율·성장률 계산  (CSV 버전 로직 그대로)
# ----------------------------------------------------------------------
growth     = (df["revenue"].iloc[-1] / df["revenue"].iloc[0])**(1/(len(df)-1)) - 1
ebit_margin = (df["ebit"] / df["revenue"]).mean()
tax_rate    = (df["tax"]  / df["ebit"].replace(0, np.nan)).mean()
dep_ratio   = (df["dep"]  / df["revenue"]).mean()
nwc_ratio   = (df["nwc_change"] / df["revenue"]).mean()

# ----------------------------------------------------------------------
# 3. 5년 Forecast → FCF
# ----------------------------------------------------------------------
future_rev = [df["revenue"].iloc[-1] * (1 + growth)**i for i in range(1, YEARS_FC + 1)]
fcf = []
for rev in future_rev:
    ebit_at = rev * ebit_margin * (1 - tax_rate)
    dep     = rev * dep_ratio
    capex   = dep                  # 유지 CapEx만 있다고 가정
    d_nwc   = rev * nwc_ratio
    fcf.append(ebit_at + dep - capex - d_nwc)

# ----------------------------------------------------------------------
# 4. DCF
# ----------------------------------------------------------------------
discounts = [(1 + WACC)**i for i in range(1, YEARS_FC + 1)]
pv_fcf = np.sum([c / d for c, d in zip(fcf, discounts)])

terminal      = fcf[-1] * (1 + G_TERMINAL) / (WACC - G_TERMINAL)
pv_terminal   = terminal / discounts[-1]

dcf_value = pv_fcf + pv_terminal

# ----------------------------------------------------------------------
# 5. 출력
# ----------------------------------------------------------------------
print("===== HISTORICAL SNAPSHOT ({} yrs) =====".format(len(df)))
print(df.T, "\n")

proj = pd.DataFrame({"Revenue": future_rev, "FCF": fcf},
                    index=[f"Y{i}" for i in range(1, YEARS_FC + 1)])
print("===== 5-YEAR PROJECTION =====")
print(proj, "\n")

print(f"📌 DCF Enterprise Value ({TICKER}):  {dcf_value:,.0f} USD")


===== HISTORICAL SNAPSHOT (4 yrs) =====
              2021-09-30    2022-09-30    2023-09-30    2024-09-30
revenue     3.658170e+11  3.943280e+11  3.832850e+11  3.910350e+11
ebit        1.089490e+11  1.194370e+11  1.143010e+11  1.232160e+11
tax         1.452700e+10  1.930000e+10  1.674100e+10  2.974900e+10
dep         1.128400e+10  1.110400e+10  1.151900e+10  1.144500e+10
nwc_change -4.911000e+09  1.200000e+09 -6.577000e+09  3.651000e+09 

===== 5-YEAR PROJECTION =====
         Revenue           FCF
Y1  3.998216e+11  1.024531e+11
Y2  4.088056e+11  1.047552e+11
Y3  4.179914e+11  1.071091e+11
Y4  4.273837e+11  1.095158e+11
Y5  4.369870e+11  1.119766e+11 

📌 DCF Enterprise Value (AAPL):  1,475,699,302,003 USD


In [41]:
##############################################################################
# 6. Net Debt, Equity Value, Intrinsic Price  ─────────────
##############################################################################
# ▸ Balance Sheet에서 총부채·현금 가져오기
bs = tk.balance_sheet.iloc[:, 0]            # 가장 최근 연도 한 열만 사용

def pick(bs_row, *labels):
    for lab in labels:
        if lab in bs.index:
            return bs[lab]
    return np.nan

total_debt = pick(bs,
                  "Total Debt",
                  "Long Term Debt",
                  "Long Term Debt & Capital Lease Obligation")     # fallback
cash_equiv = pick(bs,
                  "Cash And Cash Equivalents",
                  "Cash And Cash Equivalents At Carrying Value",
                  "Cash")

net_debt = total_debt - cash_equiv       # (–) 순현금일 수도 있음

# ▸ Equity Value = EV – Net Debt
equity_value = dcf_value - net_debt

# ▸ 주식수 (希f info가 더 최신)  – 단위: 주
shares_out = tk.get_info()["sharesOutstanding"]   # yfinance v0.2 이상 권장
# fallback: Diluted Average Shares (가장 최근 연도)
if shares_out is None:
    shares_out = grab_row(inc, "Diluted Average Shares",
                               "Basic Average Shares").iloc[0]

intrinsic_price = equity_value / shares_out

##############################################################################
# 7. 시장가격과 비교  ─────────────
##############################################################################
last_close = tk.history(period="1d")["Close"].iloc[-1]
premium_pct = (intrinsic_price - last_close) / last_close * 100

##############################################################################
# 8. 결과 출력
##############################################################################
print("===== VALUATION SUMMARY =====")
print(f"Enterprise Value (DCF)   : {dcf_value:,.0f} USD")
print(f"  – Net Debt             : {net_debt:,.0f} USD")
print(f"Equity Value             : {equity_value:,.0f} USD")
print(f"Shares Outstanding       : {shares_out:,.0f} 주")
print(f"▶ Intrinsic Value / Share: {intrinsic_price:,.2f} USD\n")

print("===== MARKET COMPARISON =====")
print(f"Last Close Price         : {last_close:,.2f} USD")
print(f"Premium / (Discount)     : {premium_pct:,.2f} %")


===== VALUATION SUMMARY =====
Enterprise Value (DCF)   : 1,475,699,302,003 USD
  – Net Debt             : 76,686,000,000 USD
Equity Value             : 1,399,013,302,003 USD
Shares Outstanding       : 14,935,799,808 주
▶ Intrinsic Value / Share: 93.67 USD

===== MARKET COMPARISON =====
Last Close Price         : 201.45 USD
Premium / (Discount)     : -53.50 %
